In [1]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("spacy_output/model-best")
nlp.add_pipe("merge_entities")

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

In [2]:
def compute_relations(patents, patterns_lhypernym, patterns_rhypernym):
  matcher = Matcher(nlp.vocab)

  matcher.add("LHypernym", patterns_lhypernym)
  matcher.add("RHypernym", patterns_rhypernym)

  doc = nlp(patents)
  matches = matcher(doc)
  print(matches)

  res = {"Hypernyms": []}

  for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]  # The matched span

    ent1 = span.ents[0]

    for word in span.ents[1:]:
      if string_id == "LHypernym":
        res["Hypernyms"].append((ent1, word))
    
      elif string_id == "RHypernym":
        res["Hypernyms"].append((word, ent1))
      
      else:
        raise ValueError(f"Unexpected match id: {string_id}")

  return res

In [3]:
patterns_rhypernym = [[{"ENT_TYPE": "TECH"},
                      {"LOWER": "include"},
                      {"LOWER": "in"},
                      {"ENT_TYPE": "TECH"}]]

patterns_lhypernym = [
                      [{"ENT_TYPE": "TECH"},
                      {"LOWER": "including"},
                      {"ENT_TYPE": "TECH"}],
                      
                      [{"ENT_TYPE": "TECH"},
                          # Optional (0 or 1 matches) of comma
                          {"OP": '?', "ORTH": ","},
                          # matches if token.lower_ == 'such'
                          {"LOWER": "such"},
                          {"LOWER": "as"},
                          {"ENT_TYPE": "TECH"}]
                     ]

compute_relations("I used Wi-Fi Direct, such as access point, Wi-Fi Alliance. Wi-Fi Direct including wireless LAN", patterns_lhypernym, patterns_rhypernym)

[(17216739248533801034, 2, 7)]


{'Hypernyms': [(Wi-Fi Direct, access point)]}

In [4]:
doc = nlp("Wi-Fi Direct (registered trademark, which will be hereinafter referred to as WFD) corresponding to a technology for directly performing a communication based on a wireless LAN between communication devices without intermediation of an access point (hereinafter referred to as AP) is standardized in Wi-Fi Alliance serving as a wireless LAN industry group.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)